In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import gradio as gr

# Sample data
data = {
    "Moisture": [30, 45, 60, 65, 70, 85, 90, 40, 55, 80, 35, 50, 60, 72, 88],
    "SoilType": [2, 2, 3, 2, 2, 1, 2, 3, 2, 1, 3, 2, 2, 2, 1],  # 1 - Sandy, 2 - Loamy, 3 - Clay
    "Fertility": [6, 7, 4, 8, 7, 5, 8, 4, 7, 5, 4, 6, 8, 7, 6],
    "Weather": [2, 3, 2, 3, 3, 1, 3, 2, 3, 1, 1, 3, 3, 3, 1]
}
df = pd.DataFrame(data)

# Fit KMeans Clustering
kmeans = KMeans(n_clusters=2, random_state=42)
df['Cluster'] = kmeans.fit_predict(df)

# Determine which cluster is "Suitable"
suitable_cluster = df.groupby("Cluster")[["Moisture", "SoilType", "Fertility", "Weather"]].mean().sum(axis=1).idxmax()

# Function to predict suitability
def check_suitability(moisture, soil_type, fertility, weather):
    input_data = np.array([[moisture, soil_type, fertility, weather]])
    cluster = kmeans.predict(input_data)[0]

    if cluster == suitable_cluster:
        return "✅ Conditions are suitable for Strawberry growth!"
    else:
        return "❌ Conditions are NOT suitable for Strawberry growth."

# Gradio interface
iface = gr.Interface(
    fn=check_suitability,
    inputs=[
        gr.Number(label="Moisture (%)"),
        gr.Dropdown(choices=[1, 2, 3], label="Soil Type (1-Sandy, 2-Loamy, 3-Clay)"),
        gr.Number(label="Fertility (1-10)"),
        gr.Dropdown(choices=[1, 2, 3], label="Weather (1-Dry, 2-Moderate, 3-Humid)")
    ],
    outputs="text",
    title="Strawberry Growth Suitability Checker (KMeans Clustering)",
    description="Enter environmental conditions to check if they are suitable for growing strawberries."
)

iface.launch()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.cluster import AgglomerativeClustering
import gradio as gr

# Sample data for clustering
data = {
    "Moisture": [30, 45, 60, 65, 70, 85, 90, 40, 55, 80, 35, 50, 60, 72, 88],
    "SoilType": [2, 2, 3, 2, 2, 1, 2, 3, 2, 1, 3, 2, 2, 2, 1],  # 1 - Sandy, 2 - Loamy, 3 - Clay
    "Fertility": [6, 7, 4, 8, 7, 5, 8, 4, 7, 5, 4, 6, 8, 7, 6],
    "Weather": [2, 3, 2, 3, 3, 1, 3, 2, 3, 1, 1, 3, 3, 3, 1]
}
df = pd.DataFrame(data)

# Fit Hierarchical Clustering (Agglomerative)
hierarchical = AgglomerativeClustering(n_clusters=2)
df['Cluster'] = hierarchical.fit_predict(df)

# Determine which cluster is "Suitable" based on centroid-like calculations
suitable_cluster = df.groupby("Cluster")[["Moisture", "SoilType", "Fertility", "Weather"]].mean().sum(axis=1).idxmax()

# Function to predict suitability
def check_suitability(moisture, soil_type, fertility, weather):
    input_data = np.array([[moisture, soil_type, fertility, weather]])
    cluster = hierarchical.fit_predict(np.vstack((df.drop(columns="Cluster").values, input_data)))[-1]

    if cluster == suitable_cluster:
        return "✅ Conditions are suitable for Strawberry growth!"
    else:
        return "❌ Conditions are NOT suitable for Strawberry growth."

# Gradio interface
iface = gr.Interface(
    fn=check_suitability,
    inputs=[
        gr.Number(label="Moisture (%)"),
        gr.Dropdown(choices=[1, 2, 3], label="Soil Type (1-Sandy, 2-Loamy, 3-Clay)"),
        gr.Number(label="Fertility (1-10)"),
        gr.Dropdown(choices=[1, 2, 3], label="Weather (1-Dry, 2-Moderate, 3-Humid)")
    ],
    outputs="text",
    title="Strawberry Growth Suitability Checker (Hierarchical Clustering)",
    description="Enter moisture, soil type, fertility, and weather conditions to check if they are suitable for strawberry growth."
)

iface.launch()


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

# Step 1: Load the housing dataset from a URL (e.g., Boston Housing dataset)
url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/housing.csv'
df = pd.read_csv(url, header=None)

# Step 2: Assign column names for clarity (assuming it's Boston Housing data)
df.columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

# Step 3: Convert housing prices into a binary classification problem
# Let's set a threshold to classify houses as 'high price' (1) or 'low price' (0)
threshold = df['MEDV'].median()
df['Price_Class'] = (df['MEDV'] > threshold).astype(int)

# Step 4: Split the data into features (X) and target (y)
X = df.drop(['MEDV', 'Price_Class'], axis=1)
y = df['Price_Class']

# Step 5: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Step 7: Train a Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Step 8: Predict on the test set
y_pred = rf_model.predict(X_test)

# Step 9: Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# Output the results
print(f'Accuracy: {accuracy*100:.2f}%')
print('Confusion Matrix:')
print(conf_matrix)